In [ ]:
%matplotlib inline

In [ ]:
!pip install torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [7]:
import numpy as np
import pandas as pd
import sklearn
import torch
import librosa
import librosa.display
import torchaudio
import os
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load metadata
metadata = pd.read_csv("/content/drive/MyDrive/UStest/UAV-velocity-prediction-master/information.csv")
metadata.head()

FileNotFoundError: ignored

In [ ]:
class AudioUtil():
    def open(audio_file):
        y, sr = librosa.load(audio_file)
        return y, sr

    # data augmentation function
    def time_shift(aud, shift_limit):
        y, sr = aud
        _, sig_len = y.shape
        shift_amt = int(random.random() * shift_limit * sig_len)
        return y.roll(shift_amt), sr
    
    def MFCCs(y, sr):
        mfccs = librosa.feature.mfcc(y=y, sr=sr)
        mfccs_scaled = np.mean(mfccs.T, axis=0)
        return mfccs_scaled

In [ ]:
from torch.utils.data import DataLoader, Dataset, random_split
import torchaudio

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, root, label):
        # file root
        self.root = root
        # slow = 0, fast = 1
        self.label = label

        fs = [os.path.join(root, f) for f in os.listdir(self.root)]
        # all file path
        self.data_files = [f for f in fs if os.path.isfile(f)]
        self.label = [label] * len(self.data_files)
    
    # __len__
    def __len__(self):
        return len(self.data_files)
    
    def __getitem__(self, idx):
        y, sr = AudioUtil.open(self.data_files[idx])
        mfcc = AudioUtil.MFCCs(y, sr)
        return mfcc, torch.tensor(self.label[idx])

In [ ]:
# file path
big_fast_path = "/content/drive/MyDrive/UStest/UAV-velocity-prediction-master/dataset/big_fast"
big_slow_path = "/content/drive/MyDrive/UStest/UAV-velocity-prediction-master/dataset/big_slow"

In [ ]:
slow_dataset = CustomDataset(big_slow_path, label = 0)
fast_dataset = CustomDataset(big_fast_path, label = 1)

slow_train, slow_valid, slow_test = torch.utils.data.random_split(slow_dataset,
[int(len(slow_dataset)*0.8), int(len(slow_dataset)*0.1), len(slow_dataset) - int(len(slow_dataset) * 0.8) - int(len(slow_dataset) * 0.1)],
generator=torch.Generator().manual_seed(42))

fast_train, fast_valid, fast_test = torch.utils.data.random_split(fast_dataset,
[int(len(fast_dataset)*0.8), int(len(fast_dataset)*0.1), len(fast_dataset) - int(len(fast_dataset) * 0.8) - int(len(fast_dataset) * 0.1)],
generator=torch.Generator().manual_seed(42))

In [ ]:
train_dataset = torch.utils.data.ConcatDataset([slow_train, fast_train])
val_dataset = torch.utils.data.ConcatDataset([slow_valid, fast_valid])
test_dataset = torch.utils.data.ConcatDataset([slow_test, fast_test])

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
val_loader = DataLoader(val_dataset, batch_size=16)

In [ ]:
import torch.nn as nn

In [ ]:
import torch
import torch.nn as nn

# Neural Networks example
class ClassifireNN(nn.Module):
    def __init__(self, drop_out=0.0):
        super(ClassifireNN, self).__init__()
        self.fc1 = nn.Linear(20, 32)
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, 8)
        self.fc4 = nn.Linear(8, 1)
        
        self.relu = nn.ReLU()

        self.drop_out = nn.Dropout(p=drop_out)

    def forward(self, x):
        # x = x.view(-1, 20)  

        x = self.relu(self.fc1(x))
        x = self.drop_out(x)
        x = self.relu(self.fc2(x))
        x = self.drop_out(x)
        x = self.relu(self.fc3(x))
        x = self.fc4(x)

        x = torch.sigmoid(x)

        return x.view(-1)

# CNN example
class ClassifireCNN(nn.Module):
    def __init__(self, drop_out=0.0):
        super(ClassifireCNN, self).__init__()
        self.cnn1 = nn.Conv1d(in_channels=20, out_channels=64, kernel_size=5, padding=2)
        self.cnn2 = nn.Conv1d(in_channels=64, out_channels=8, kernel_size=5, padding=2)
        self.cnn3 = nn.Conv1d(in_channels=8, out_channels= 1, kernel_size=5, padding=2)


        # self.pool1 = nn.MaxPool1d(3)
        # self.pool1 = nn.MaxPool1d(4)
        # self.pool1 = nn.MaxPool1d(5)

        # self.fc1 = nn.Linear(4 * 128, 512)
        # self.fc2 = nn.Linear(512, 128)
        # self.fc3 = nn.Linear(128, 64)
        # self.fc4 = nn.Linear(64, 32)
        # self.fc5 = nn.Linear(32, 1)
        
        self.relu = nn.ReLU()

        self.drop_out = nn.Dropout(p=drop_out)

    def forward(self, x):
        # input : [16, 20], [batch, feature]
        x = torch.reshape(x, (-1, 20, 1))   #[batch, feature, 1]
        x = self.relu(self.cnn1(x))         # [batch, 8, 1]
        x = self.relu(self.cnn2(x))         # [batch, 4, 1]
        x = self.cnn3(x)                    # [batch, 1, 1]   

        x = torch.sigmoid(x)

        return x.view(-1)


In [ ]:
device = torch.device(f'cuda:0' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
LR = 0.0001
PATIENCE = 3
FACTOR = 0.95
DROP_OUT = 0.3
EPOCHS = 100

In [ ]:
model = ClassifireCNN(drop_out=DROP_OUT).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
criterion = nn.BCELoss()

In [ ]:
best_auc = 0
best_epoch = -1
best_pred = []

prev_model = None

In [ ]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from torch.utils.tensorboard import SummaryWriter

In [ ]:
for i in tqdm(range(EPOCHS)):

    # Train
    loss_sum = 0
    train_loss_sum = 0
    val_loss_sum = 0
    true_labels = []
    pred_labels = []
    t_true_labels = []
    t_pred_labels = []
    v_true_labels = []
    v_pred_labels = []
    model.train()

    for e_num, (x, y) in enumerate(train_loader):

        x, y = x.type(torch.FloatTensor).to(device), y.type(torch.FloatTensor).to(device)
        
        model.zero_grad()
        pred_y = model(x)

        tloss = criterion(pred_y, y)
        writer.add_scalar("Loss/train", tloss, i)
        train_loss_sum += tloss.detach()
        optimizer.zero_grad()
        tloss.backward()
        optimizer.step()

        true_labels.extend(y.cpu().numpy())
        pred_labels.extend(np.around(pred_y.cpu().detach().numpy()))
        t_true_labels.extend(y.cpu().numpy())
        t_pred_labels.extend(np.around(pred_y.cpu().detach().numpy()))

    auc = accuracy_score(true_labels, pred_labels)
    train_auc = accuracy_score(t_true_labels, t_pred_labels)
    loss_sum = train_loss_sum
    loss = tloss
    # Valid
    for e_num, (x, y) in enumerate(val_loader):
        x, y = x.type(torch.FloatTensor).to(device), y.type(torch.FloatTensor).to(device)

        pred_y = model(x)
        vloss = criterion(pred_y, y)
        loss = vloss

        val_loss_sum += vloss.detach()
        writer.add_scalar("Loss/val", vloss, i)
        loss_sum += loss.detach()
        true_labels.extend(y.cpu().numpy())
        pred_labels.extend(np.around(pred_y.cpu().detach().numpy()))
        v_true_labels.extend(y.cpu().numpy())
        v_pred_labels.extend(np.around(pred_y.cpu().detach().numpy()))

    auc = accuracy_score(true_labels, pred_labels)
    val_auc = accuracy_score(v_true_labels, v_pred_labels)
    
    # wirter.add_scalar("")

    if auc > best_auc:
        best_pred = pred_labels
        best_auc = auc
        best_epoch = i

        if prev_model is not None:
            os.remove(prev_model)
        prev_model = f'cnn_model_{best_auc}.h5'
        torch.save(model.state_dict(), prev_model)

    if i % 5 == 4:
      print(f"best validation acc = {best_auc}, in epoch {best_epoch}")
      print("loss = ", loss_sum)
      print("train_loss = ", train_loss_sum, "val_loss", val_loss_sum)
      print("auc = ", auc)
      print("train_auc = ", train_auc, "val_auc = ", val_auc)

print(f"best validation acc = {best_auc}, in epoch {best_epoch}")
writer.flush()
writer.close()

In [ ]:
$ ssh -NfL localhost:16006:localhost:6006 sjo55

In [ ]:
!pip install tensorboard

In [ ]:
!tensorboard --logdir=runs